In [16]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
from proj.cxs_are_revealed.paper.data_config import Exp2Cogs
# overall plan

# dataset
# - label which indexes

# model -> get fills at position (deal with multitoken)
# get likely fills + probability mass
# sum probability mass on comp adjectives

#

In [49]:
# read in cogs data for comp corr
# note that we already processed in exp8_cogs
from proj.cxs_are_revealed.paper.cxns_in_distrib.exp2_cogs import read_csv_row_by_row, get_all_data_clean, CogsEntry

csv_data_clean = read_csv_row_by_row(Exp2Cogs.cogs_parsed_final)
print(csv_data_clean[0])
all_cogs_clean = get_all_data_clean(csv_data_clean, fix_punct_in_comp_corr=False)

['Let Alone', 'Most wives are too bloody old, let alone mothers.', '[6, 7]', "[(0, 'Most'), (1, 'wives'), (2, 'are'), (3, 'too'), (4, 'bloody'), (5, 'old,'), (6, 'let'), (7, 'alone'), (8, 'mothers.'), (9, '(FN'), (10, 'Construction)')]", '']
in sent Though punctuated by frequent flash-backs to the period before, during and just after the war, temporal progression in the present is clearly marked by the development of two narrative lines which weave their ways in and out of the novel., idx 33 != way
note expected one error message bc one word is 'ways' instead of 'way'


In [50]:
comp_corrs = [c for c in all_cogs_clean if c.cx_type == 'Comparative Correlative' ]
print(len(comp_corrs))
print(comp_corrs[0])

54
CogsEntry(id=156, cx_type='Comparative Correlative', sent='The harder they come, the harder they fall.', tgt_words=[0, 4], tgt_word_offsets=[(0, 3), (22, 25)])


In [51]:
from proj.cxs_are_revealed.paper.cxns_in_distrib.exp5_cc.exp9_cc_utils import is_comparative

is_comparative("harder")

True

In [53]:
from proj.cxs_are_revealed.paper.cxns_in_distrib.exp5_cc.exp9_cc_utils import check_cc

# check that after each "the" we do in fact have a comparative
# this is also checked below; we wrote code twice to extract the comparative adjectives
def check_all():
    for cc in comp_corrs:
        check_cc(cc)
check_all()



In [54]:
from lib.common.mlm_singleton import init_singleton_scorer
mlm_scorer = init_singleton_scorer('roberta-large', output_attentions=False)



WARN: singleton already initialized; will not re-init


In [55]:
from dataclasses import dataclass
from typing import List, Tuple
from transformers import PreTrainedTokenizerFast

# todo
# gpt o1 generated; probably duplicates some of our code somewhere else
# goes from a sentence to list of words with char offsets
@dataclass
class WordAlignment:
    word: str
    word_idx: int
    token_idxs: List[int]
    tokens: List[str]
    char_offset: Tuple[int, int]

def get_word_token_alignments(sentence: str, tokenizer: PreTrainedTokenizerFast) -> List[WordAlignment]:
    """
    Splits `sentence` on spaces and obtains token alignments via the provided fast tokenizer.
    Returns a list of WordAlignment objects with character offsets, token indices, and token strings.
    """
    encoding = tokenizer(sentence, return_offsets_mapping=True, add_special_tokens=False)
    offsets = encoding["offset_mapping"]
    token_ids = encoding["input_ids"]
    tokens = [tokenizer.convert_ids_to_tokens(t_id) for t_id in token_ids]

    word_alignments: List[WordAlignment] = []
    words = sentence.split(" ")

    current_char_pos = 0
    token_idx = 0

    for w_idx, word in enumerate(words):
        start_char = current_char_pos
        end_char = start_char + len(word)

        # Accumulate tokens that fully map to the current word
        current_token_indices: List[int] = []
        current_tokens: List[str] = []

        while token_idx < len(offsets):
            t_start, t_end = offsets[token_idx]
            if t_start >= start_char and t_end <= end_char:
                current_token_indices.append(token_idx)
                current_tokens.append(tokens[token_idx])
                token_idx += 1
            else:
                # If token is beyond the current word boundary, stop checking
                if t_end > end_char:
                    break
                token_idx += 1

        word_alignments.append(
            WordAlignment(
                word=word,
                word_idx=w_idx,
                token_idxs=current_token_indices,
                tokens=current_tokens,
                char_offset=(start_char, end_char)
            )
        )

        current_char_pos = end_char + 1  # move past the space

    return word_alignments

sent = "This is a test sentence."
get_word_token_alignments(sent, mlm_scorer.tokenizer)


[WordAlignment(word='This', word_idx=0, token_idxs=[0], tokens=['This'], char_offset=(0, 4)),
 WordAlignment(word='is', word_idx=1, token_idxs=[1], tokens=['Ġis'], char_offset=(5, 7)),
 WordAlignment(word='a', word_idx=2, token_idxs=[2], tokens=['Ġa'], char_offset=(8, 9)),
 WordAlignment(word='test', word_idx=3, token_idxs=[3], tokens=['Ġtest'], char_offset=(10, 14)),
 WordAlignment(word='sentence.', word_idx=4, token_idxs=[4, 5], tokens=['Ġsentence', '.'], char_offset=(15, 24))]

In [56]:
from proj.cxs_are_revealed.paper.cxns_in_distrib.exp3_magpie import WrappedIdiomWord, EntryForProcessing, set_allow_deprecated
from transformers import BatchEncoding
from typing import List, Tuple
from dataclasses import dataclass, field


@dataclass
class CCEx:
    id: int
    sent: str
    comp_words_idxs: List[int]
    # todo: indexes or words

# below adapted from exp8_cogs (which was in turn prev adapted from sth else)
@dataclass
class CCExOneWord:
    # original sentence id in dataset
    id: int = field(init=False)
    sent: str = field(init=False)

    # will be only one for each to make it simple
    tgt_word_offsets: List[Tuple[int, int]] = field(default_factory=list)
    words: List[str] = field(init=False)

set_allow_deprecated()

class Wrapper:
    """ Adapted from MAGPIE_Wrapper"""
    def __init__(self, cog_entry: CCExOneWord):
        """
        Steps:
        - get the total wordlist (contiguous non space strings)
        - get the alphabetic wordlist (candidates for masking)
        - get the list of idiom words (those with offsets)
            (checks that idiom words do not have spaces in them)

        """
        self.cog_entry = cog_entry

        encoding = mlm_scorer.get_batch_encoding_for_sentence(cog_entry.sent)

        # idiom token indices
        self.idiom_token_list: List[WrappedIdiomWord] = self.get_idiom_token_list(encoding)

        # later (other methods)
        # produce a list of possible tokens / words to mask from a sentence

    def get_idiom_token_list(self, encoding: BatchEncoding):
        offset_to_token_list_map: List[WrappedIdiomWord] = []
        for offset in self.cog_entry.tgt_word_offsets:
            wrapped_idiom = WrappedIdiomWord.wrap_idiom_using_encoding(self.cog_entry, offset, encoding)
            offset_to_token_list_map.append(wrapped_idiom)

        assert len(offset_to_token_list_map) == len(self.cog_entry.tgt_word_offsets)
        return offset_to_token_list_map

def get_all_data_clean(all_comp: List[CogsEntry]):
    ret_list: List[CCExOneWord] = []
    for idx, cc_sent in enumerate(all_comp):
        sent_words = cc_sent.sent.split(" ")
        # +1 to go from "the" => comparative adj
        comparative_words = [sent_words[i+1] for i in cc_sent.tgt_words]
        # assert(all([is_comparative_with_context(cc_sent.sent, offset) for offset in cc_sent.tgt_word_offsets]))

        sent_word_alignments = get_word_token_alignments(cc_sent.sent, mlm_scorer.tokenizer)
        tgts = [sent_word_alignments[i + 1] for i in cc_sent.tgt_words]
        for t, w in zip(tgts, comparative_words):
            assert t.word == w

            cc_ex = CCExOneWord()
            ret_list.append(cc_ex)
            cc_ex.id = cc_sent.id
            cc_ex.sent = cc_sent.sent
            cc_ex.tgt_word_offsets = [t.char_offset]
            cc_ex.words = [w]

            assert cc_sent.sent[t.char_offset[0]:t.char_offset[1]] == w

            # make sure original is comparative
            assert is_comparative_with_context(cc_sent.sent, t.char_offset)

    return ret_list

all_cc_exs = get_all_data_clean(comp_corrs)
len(all_cc_exs)



108

In [57]:
all_wrapped = [Wrapper(c) for c in all_cc_exs]
len(all_wrapped)
all

<function all(iterable, /)>

In [58]:
print(all_wrapped[0].idiom_token_list)
print(all_wrapped[0].cog_entry.sent)

[WrappedIdiomWord(offset=(4, 10), idiom_word_chars='harder', tokens=[2], token_words=['harder'], is_multiple_tokens=False, tokens_do_not_exact_match=False)]
The harder they come, the harder they fall.


In [60]:
import torch


# will calculate the total probability mass for comparative adjectives
# 1. order by probability
# get prob scores
# while total prob is < value, check the next word
# sum
def score_distrib(cog_entry: CCExOneWord, logits: torch.Tensor, top_p=0.98) -> float:
    # get sorted
    logit_indices_sorted = torch.argsort(logits, descending=True)
    probs = torch.softmax(logits, dim=-1)
    total_prob_mass = 0
    comparative_prob_mass = 0
    idx = 0
    while total_prob_mass < top_p:
        if idx > 100:
            print(f"idx is at 100 for {cog_entry.sent}")
            break
        # loop iteration
        orig_logit_idx = logit_indices_sorted[idx]
        idx += 1    # increment for next
        p = probs[orig_logit_idx]
        total_prob_mass += p

        # also accumulate in our tracker if it's comparative
        word = mlm_scorer.tokenizer.decode([orig_logit_idx]).strip(" ") # generally has a space
        orig_sent = cog_entry.sent
        sent_with_word = (
                orig_sent[:cog_entry.tgt_word_offsets[0][0]] +
                word +
                orig_sent[cog_entry.tgt_word_offsets[0][1]:])
        # print(orig_sent, "=>\n\t", sent_with_word)
        new_offset_start = cog_entry.tgt_word_offsets[0][0]
        new_offset_end = new_offset_start + len(word)
        if is_comparative_with_context(sent_with_word, (new_offset_start, new_offset_end)):
            comparative_prob_mass += p
        # else:
        #     print(f"In {cog_entry.sent} \n\tnew word {word} not comparative")

    return comparative_prob_mass/ total_prob_mass

# test that it does something else for an invalid one
def test_distrib():
    # sent = "I want to get better at this game."
    # sent = "It is worth noting , however , that the more specific you are the better ."
    sent =  "The higher up the nicer !"
    sent =  "The bigger two fight the smaller cats."
    sent =  "The the two cats fight, the louder."
    sent = "The harder the two cats fight ....."
    sent = "The harder two fight the cats ....."
    # sent = "The harder the two cats fight."
    sent = "The harder two fight the cats."

    # sent = "This is a test."

    sent_word_alignments = get_word_token_alignments(sent, mlm_scorer.tokenizer)
    tgt = sent_word_alignments[1]
    print(tgt)
    cc_test = CCExOneWord()
    # cc_test.id = 0
    cc_test.id = 0
    cc_test.sent = sent
    cc_test.tgt_word_offsets = [tgt.char_offset]
    cc_test.words = ["harder"]
    # print(cc_test)
    w = Wrapper(cc_test)
    # print(w.idiom_token_list)
    score = process_sent_exp8(w)
    print(score)


    # assert len(cog_entry_wrapped.idiom_token_list) == 1 # we put only one in each
    # word_to_mask = cog_entry_wrapped.idiom_token_list[0]
test_distrib()






WordAlignment(word='harder', word_idx=1, token_idxs=[1], tokens=['Ġharder'], char_offset=(4, 10))
idx is at 100 for The harder two fight the cats.
tensor(0.0162, device='mps:0')


In [61]:
from typing import Optional


# copied from exp8_cogs
def process_sent_exp8(
        cog_entry_wrapped: Wrapper
) -> Optional[float]:
    cog_entry = cog_entry_wrapped.cog_entry
    entry_for_processing = EntryForProcessing(cog_entry.id, cog_entry.sent)

    assert len(cog_entry_wrapped.idiom_token_list) == 1 # we put only one in each
    word_to_mask = cog_entry_wrapped.idiom_token_list[0]
    # print(word_to_mask)

    if word_to_mask.is_multiple_tokens:
        print(f"Multi token: {word_to_mask}\n {cog_entry.sent}")
        return None

    assert word_to_mask.tokens_do_not_exact_match is False, f"{word_to_mask}"


    t_idx = word_to_mask.tokens[0]
    inputs_ids = entry_for_processing.get_inputs_with_mask_for_token_at_idx(
        t_idx)
    # get likely fills
    outputs = mlm_scorer.get_model_outputs_for_input(inputs_ids)
    logits = outputs.logits

    # predictions is vocab_len [logit, logit, ... logit]
    token_logits = logits[0, t_idx]     # batch, idx, then vocab_len shape

    # orig_token_id = entry_for_processing.input_ids[0, t_idx]
    score = score_distrib(cog_entry, token_logits)

    # print(cog_entry.sent, word_to_mask.idiom_word_chars)
    # print(score)

    return score

def get_all_scores():
    all_scores: List[float] = []
    # for wrapped in tqdm(all_wrapped):
    for wrapped in all_wrapped:
        score = process_sent_exp8(wrapped)
        all_scores.append(score)
    return all_scores

all_scores = get_all_scores()


Multi token: WrappedIdiomWord(offset=(13, 21), idiom_word_chars='merrier.', tokens=[4, 5, 6], token_words=['mer', 'rier', '.'], is_multiple_tokens=True, tokens_do_not_exact_match=False)
 The more the merrier.
Multi token: WrappedIdiomWord(offset=(29, 37), idiom_word_chars='better,"', tokens=[9, 10], token_words=['better', ',"'], is_multiple_tokens=True, tokens_do_not_exact_match=False)
 " The more that is done, the better," he told reporters, while promising more money for disaster preparedness.
Multi token: WrappedIdiomWord(offset=(29, 36), idiom_word_chars='better.', tokens=[7, 8], token_words=['better', '.'], is_multiple_tokens=True, tokens_do_not_exact_match=False)
 The more wealth created, the better.
Multi token: WrappedIdiomWord(offset=(53, 60), idiom_word_chars='better.', tokens=[15, 16], token_words=['better', '.'], is_multiple_tokens=True, tokens_do_not_exact_match=False)
 All I know, is the more I see her on television, the better.
Multi token: WrappedIdiomWord(offset=(53, 5

# Analysis

In [63]:
print(len(all_scores)) # 108, as expected, 54*2

# 9 failed
print(len([s for s in all_scores if s is None]))

# how many have less than perfect score? - 4
print(len([s for s in all_scores if s is not None and s < 1]))

# look at the ones with score less than 100% (there are 4)
low = [(i, s) for i, s in enumerate(all_scores) if s is not None and s < 1]
for i,s in low:
    print(all_cc_exs[i])
    print(s)


108
9
4
CCExOneWord(id=190, sent='The higher up the nicer !', tgt_word_offsets=[(18, 23)], words=['nicer'])
tensor(0.8603, device='mps:0')
CCExOneWord(id=193, sent='It is worth noting , however , that the more specific you are the better .', tgt_word_offsets=[(66, 72)], words=['better'])
tensor(0.9961, device='mps:0')
CCExOneWord(id=199, sent='The more of them you see, the cheaper it is.', tgt_word_offsets=[(30, 37)], words=['cheaper'])
tensor(0.9961, device='mps:0')
CCExOneWord(id=202, sent='The sooner it is tested, the better.', tgt_word_offsets=[(4, 10)], words=['sooner'])
tensor(0.9903, device='mps:0')
